# Python Notebook to Figure Out .vts Files

Brandon Hardy

In [1]:
# imports
import pyvista as pv
import numpy as np
import nrrd

## Putting MR Angiography Images in Correct Coordinate System

In [2]:
# Load the NRRD file
filename = "test/603 MRA Last Phase.nrrd"
data, header = nrrd.read(filename)

# Extract space dimensions
space_dimensions_MRA = header['space directions'].T
space_origin = header['space origin']
data_size = header['sizes']

In [3]:
# we start with a meshgrid where each position in the meshgrid has an integer value (0, 1, 2, ...), since
# the nrrd header specifies how space changes as we increment the unit
# vectors by 1. I'm not sure if this is how other headers operate (nifti, DICOM),
# but this works for nrrd files

xv = np.arange(0, data_size[0] + 1, dtype=np.float32) # add 1 because we need point locations for pyvista
yv = np.arange(0, data_size[1] + 1, dtype=np.float32)
zv = np.arange(0, data_size[2] + 1, dtype=np.float32)

X, Y, Z = np.meshgrid(xv, yv, zv, indexing='ij')

In [4]:
# ravel to 1D and transform all coordinates at once
coords = np.vstack((X.ravel(), Y.ravel(), Z.ravel()))
transformed_coords = np.dot(space_dimensions_MRA, coords) + space_origin[:, np.newaxis]

X = transformed_coords[0, :].reshape(X.shape)
Y = transformed_coords[1, :].reshape(Y.shape)
Z = transformed_coords[2, :].reshape(Z.shape)

In [5]:
# convert coordinates to float32 to save space
X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)
Z = np.asarray(Z, dtype=np.float32)

In [6]:
grid = pv.StructuredGrid(X, Y, Z)
grid.cell_data["Image Intensity"] = data.flatten(order="F")
grid.field_data["Space Origin"] = space_origin
plane_normals = np.linalg.inv(space_dimensions_MRA)
grid.field_data["Image Axis 1"] = plane_normals[0] # I don't really know why I'm inverting this but it gives the right result if you want to draw planes along the axial, saggital, and coronal planes
grid.field_data["Image Axis 2"] = plane_normals[1]
grid.field_data["Image Axis 3"] = plane_normals[2]
grid.save("603_MRA_Last_Phase.vts")

## Putting 4D Flow Images in Correct Coordinate System

In [10]:
# Set up paths and read in 4D flow data and segmentation masks
phase_directions = ["RL", "AP", "FH"]
paths = [f"test/{direction}_4D/{903+i} MR DelRec - 4D PC_{direction} - 24 frames Volume Sequence by TriggerTime" for i, direction in enumerate(phase_directions)]

flow_header = nrrd.read_header(paths[0] + " 0.nrrd")
space_dimensions_4Dflow = flow_header['space directions'].T
space_origin = flow_header['space origin']
data_size = flow_header['sizes']

mask_TL = nrrd.read("test/Segmentation_TL_4DFLOWspace.nrrd")[0]
mask_FL = nrrd.read("test/Segmentation_FL_4DFLOWspace.nrrd")[0]

In [11]:
xv = np.arange(0, data_size[0] + 1, dtype=np.float32)
yv = np.arange(0, data_size[1] + 1, dtype=np.float32)
zv = np.arange(0, data_size[2] + 1, dtype=np.float32) # plus 1 because we need point locations

X, Y, Z = np.meshgrid(xv, yv, zv, indexing='ij')

# ravel to 1D and transform all coordinates at once
coords = np.vstack((X.ravel(), Y.ravel(), Z.ravel()))
transformed_coords = np.dot(space_dimensions_4Dflow, coords) + space_origin[:, np.newaxis]

X = transformed_coords[0, :].reshape(X.shape)
Y = transformed_coords[1, :].reshape(Y.shape)
Z = transformed_coords[2, :].reshape(Z.shape)

X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)
Z = np.asarray(Z, dtype=np.float32)

In [12]:
# adapted from a function in a different set of code
def import_flow(
    paths: tuple[str, str, str],
    timestep: float,
) -> np.ndarray:

    vels = []
    
    for i, path in enumerate(paths):
        # read in the nrrd file
        data = nrrd.read(f"{path} {timestep}.nrrd")[0]
        data[data < -199] = 0  # set weird cropped values to 0
        data = data/100 # convert to m/s
        
        vels.append(data)
        
    return vels

In [ ]:
# 24 frames of 4D flow data
# normalize the columns since we aren't going to scale the velocity data -- just change the orientation
space_dimensions_vel = np.zeros((3, 3), dtype=np.float32)
space_dimensions_vel[:, 0] = space_dimensions_MRA[:, 0] / np.linalg.norm(space_dimensions_MRA[:,0])
space_dimensions_vel[:, 1] = space_dimensions_MRA[:, 1] / np.linalg.norm(space_dimensions_MRA[:,1])
space_dimensions_vel[:, 2] = space_dimensions_MRA[:, 2] / np.linalg.norm(space_dimensions_MRA[:,2])

# absolutely no idea why the MRA space works for the velocity vectors but it does... probably a weird coincidence
# anyway I'm taking norm of each column to turn the matrix into a rotation matrix -- no scaling

paths_copy = paths.copy()
#paths[0] = paths_copy[2] # 0, 0, 1, 1, 2
#paths[1] = paths_copy[0] # 1, 2, 0, 2, 1
#paths[2] = paths_copy[1] # 2, 1, 2, 0, 0
for nt in range(24):
    vel_t = import_flow(paths, nt)
    if nt == 1:
        print(vel_t[0].shape)

    for i in range(3):
        vel_t[i] = vel_t[i] * (mask_FL + mask_TL !=0)  # combine the masks
        vel_t[i] = vel_t[i].flatten(order="F")
    
    # can't use dimensions exactly... need to orthonormalize?
    vels = np.vstack((vel_t[0], vel_t[1], vel_t[2]))
    transformed_vels = np.dot(space_dimensions_vel, vels)
    
    grid = pv.StructuredGrid(X, Y, Z)
    grid.cell_data.set_vectors(transformed_vels.T, "Velocity")  # Assign velocity vectors to point data
    grid.save(f"4DFlow_{nt:02d}.vts")

(288, 288, 40)


KeyboardInterrupt: 

In [ ]:
# Set numpy print options to suppress scientific notation
with np.printoptions(suppress=True, precision=6):
    print(f"MRA conversion matrix: \n{space_dimensions_MRA}\n")
    print(f"4D flow conversion matrix: \n{space_dimensions_4Dflow}")

MRA conversion matrix: 
[[ 0.845658  0.520558  0.103086]
 [-0.524349  0.851501  0.001216]
 [-0.099594 -0.06295   0.868906]]

4D flow conversion matrix: 
[[ 0.617612 -0.139777 -2.114147]
 [ 1.010257 -0.001649  1.310873]
 [-0.074686 -1.178177  0.248985]]


In [15]:
data.shape

(96, 261, 400)

In [16]:
vel_t[0].shape

(3317760,)